<h2 style="color: red;"><strong>Imports:</strong></h2>

Qua importiamo le librerie che utilizzeremo nel corso del nostro programma

In [241]:
import pandas as pd

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

<h2 style="color: red;"><strong>Data Analysis:</strong></h2>

<h4 style="color: gold;">Funzioni:</h4>

La funzione stampaPercentDF() ha come scopo quello di restituire le percentuali della colonna Severity, passandogli come parametro il dataframe su cui vogliamo vedere le percentuali

In [242]:
def stampaPercentDF(df):
    for name, group in df:
    # La funzione value_counts() ritorna la percentuale di apparizione per ogni elemento univoco,
    # è come un groupby però ci aggiunge le percentuali (se aggiungo il parametro normalize) altrimenti ritornebbe solo il numero di volte in cui i valori appaiono
        group = round(group['Severity'].value_counts(normalize=True) * 100, 2)
        print(name)
        print(group)
        print('-'*10)

<h4 style="color: gold;">DataFrames:</h4>

In [243]:
df_completo = pd.read_csv('CSVCompleto.csv')

<p style = "color: gray;">Elimino tutti i valori Nan per evitare eventuali errori.</p>

In [244]:
df_completo = df_completo[~(pd.isna(df_completo.iloc[:,0]))]

<p style = "color: gray;">Drop di colonne / righe non utili allo studio dei dati o che rendono più complesso lo studio</p>

Abbiamo scelto di eliminare tutte le righe che avevano nella colonna Type il valore Moving average, che simboleggiava un arco temporale, abbiamo deciso di escludere questi valori per aiutarci ad analizzare i dati anno per anno per una maggior accuratezza.

In [245]:
df_completo = df_completo.drop(df_completo[df_completo['Type'] == 'Moving average'].index, axis=0)

Con questo codice, eliminiamo le colonne che contengono dati che abbiamo decretato non essere utili per l'analisi.
La decisione di quali colonne è stata fatte tramite l'ausilio della funzione stampaPercentDF() che riportava, per tutte queste colonne, o percentuali simili oppure elementi troppo specifici:

In [246]:
print('------ Period ------')
stampaPercentDF(df_completo.groupby('Period'))
print('------ Data Value ------')
stampaPercentDF(df_completo.groupby('Data_value'))
print('------ Lower_CI ------')
stampaPercentDF(df_completo.groupby('Lower_CI'))
print('------ Upper_CI ------')
stampaPercentDF(df_completo.groupby('Upper_CI'))
print('------ Units ------')
stampaPercentDF(df_completo.groupby('Units'))
print('------ Indicator ------')
stampaPercentDF(df_completo.groupby('Indicator'))
print('------ Cause ------')
stampaPercentDF(df_completo.groupby('Cause'))
print('------ Validation ------')
stampaPercentDF(df_completo.groupby('Validation'))
print('------ Population ------')
stampaPercentDF(df_completo.groupby('Population'))
print('------ Age ------')
stampaPercentDF(df_completo.groupby('Age'))

------ Period ------
2000
Serious              40.38
Serious non-fatal    30.77
Fatal                28.85
Name: Severity, dtype: float64
----------
2001
Serious              38.89
Fatal                31.48
Serious non-fatal    29.63
Name: Severity, dtype: float64
----------
2002
Serious              37.93
Serious non-fatal    31.03
Fatal                31.03
Name: Severity, dtype: float64
----------
2003
Serious              37.93
Serious non-fatal    31.03
Fatal                31.03
Name: Severity, dtype: float64
----------
2004
Serious              37.93
Serious non-fatal    31.03
Fatal                31.03
Name: Severity, dtype: float64
----------
2005
Serious              37.93
Serious non-fatal    31.03
Fatal                31.03
Name: Severity, dtype: float64
----------
2006
Serious              37.93
Serious non-fatal    31.03
Fatal                31.03
Name: Severity, dtype: float64
----------
2007
Serious              37.93
Serious non-fatal    31.03
Fatal                31.

<p> Dopo un'attenta analisi delle percentuali si evince che le colonne che non hanno prodotto percentuali soddisfacenti sono:

In [247]:
df_completo = df_completo.drop(['Series_reference', 'Validation', 'Indicator', 'Type'], axis=1)

<p style = "color: gray;">Raggruppo e faccio il mapping di dati, dato che scikit-learn non ammette stringhe nei suoi modelli</p>

Qui appunto eseguo il mapping in base ai dati che sono di tipo stringa, attribuendogli un valore intero.
Ad ogni colonna ho dedicato una map su cui ho indicizzato tutti i valori che contiene la colonna, col il rispettivo sostituto intero.

Per trovare quali valori contenesse ogni colonna abbiamo fatto un semplice groupby della colonna e successivamente stampato tutti i valori raggruppati

Es: con la colonna Units

In [248]:
df_esempio = df_completo.groupby('Units')
df_esempio.all()

,Period,Data_value,Lower_CI,Upper_CI,Cause,Population,Age,Severity
Units,,,,,,,,
Injuries,True,True,True,True,True,True,True,True
"Per 100,000 FTEs",True,True,True,True,True,True,True,True
"Per 100,000 people",True,True,True,True,True,True,True,True
Per billion km,True,True,True,True,True,True,True,True
Per thousand registered vehicles,True,True,True,True,True,True,True,True


In [249]:
units_map = {
    'Injuries': 1,
    'Per 100,000 FTEs': 2,
    'Per 100,000 people': 3,
    'Per billion km': 4,
    'Per thousand registered vehicles': 5
}
df_completo['Units'] = df_completo['Units'].map(units_map)

pop_map = {
    'Maori': 1,
    'Whole pop': 2,
    'Children': 3
}
df_completo['Population'] = df_completo['Population'].map(pop_map)

cause_map = {
    'All': 1,
    'Assault': 2,
    'Drowing': 3,
    'Falls': 4,
    'Intentional self-harm': 5,
    'Motor vehicle traffic crashes': 6,
    'Work': 7,
    'Car occupant': 8,
    'Intentional': 9,
    'Pedestrian': 10
}
df_completo['Cause'] = df_completo['Cause'].map(cause_map)

age_map = {
    '0-14 years': 1,
    '0-74 years': 2,
    '75+ years': 3,
    'All ages':4
}
df_completo['Age'] = df_completo['Age'].map(age_map)

severity_map = {
    'Fatal': 1,
    'Serious non-fatal': 2,
    'Serious': 3
}
df_completo['Severity'] = df_completo['Severity'].map(severity_map)

# Arrondo i numeri a 2 cifre dopo la virgola, dato che nel CSV ne hanno molti di più.
df_completo['Lower_CI'] = round(df_completo['Lower_CI'], 2)
df_completo['Upper_CI'] = round(df_completo['Upper_CI'], 2)

print(df_completo)

     Period  Data_value  Lower_CI  Upper_CI  Units  Cause  Population  Age  \
0      2017  345.000000    308.60    381.40      1      4           1    2   
1      2018  372.000000    334.20    409.80      1      4           1    2   
2      2000   34.629659     29.76     39.50      3      4           1    2   
3      2001   32.213739     27.57     36.86      3      4           1    2   
4      2002   30.495950     26.03     34.97      3      4           1    2   
...     ...         ...       ...       ...    ...    ...         ...  ...   
2743   2012  255.000000    223.70    286.30      1      4           1    2   
2744   2013  294.000000    260.39    327.61      1      4           1    2   
2745   2014  275.000000    242.50    307.50      1      4           1    2   
2746   2015  273.000000    240.62    305.38      1      4           1    2   
2747   2016  323.000000    287.78    358.22      1      4           1    2   

      Severity  
0            3  
1            3  
2           

<h2 style="color: Red;"> Machine Learning</h2>

<p style="color: gray;">Dividiamo il file csv nei file di train e di test, il file di test ha il 30% del totale dei dati, mentre il file di train ha il 70% restante</p>

In [250]:
# suddivide il dataframe in 70/30
df_test, df_train = train_test_split(df_completo, test_size=0.3, random_state=42)

# salva i dati in due file CSV distinti
df_train.to_csv('medical-test.csv', index=False)
df_test.to_csv('medical-train.csv', index=False)

<p style = "color: gray">Creo le variabili di test / train utili per la predizione dei dati nei vari modelli</p>

In [251]:
Y_test = df_test["Severity"]
Y_train = df_train["Severity"]
X_train = df_train.drop("Severity", axis=1)
X_test  = df_test.drop("Severity", axis=1,errors='ignore').copy()
X_train.shape, Y_train.shape, X_test.shape

((690, 8), (690,), (1608, 8))

<h5 style="color: pink;">Logistic Regression</h5>

In [252]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
print(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
print(acc_log)
acc_log = round(logreg.score(X_test, Y_test) * 100, 2)

acc_log

     Period  Data_value  Lower_CI  Upper_CI  Units  Cause  Population  Age
1341   2014  288.000000    254.74    321.26      1      6           2    4
352    2015    1.616249      1.26      1.97      3      4           2    2
91     2009   11.567471      9.37     13.76      3      1           3    1
2317   2001   48.104630     42.47     53.74      3      1           1    4
2285   2009   22.271587     20.16     24.38      2      7           2    4
...     ...         ...       ...       ...    ...    ...         ...  ...
2088   2000  109.994046    106.63    113.36      3      4           2    4
1545   2001   12.021858     10.90     13.15      4      6           2    4
1580   2018    0.891223      0.80      0.98      5      6           2    4
1744   2020   14.994481     13.91     16.08      3      2           2    4
1310   2002    3.034495      2.49      3.58      3      3           2    4

[1608 rows x 8 columns]
45.07


C:\Users\marko\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


45.09

<h5 style="color: pink;">Support Vector Machines</h5>

In [253]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

39.28

<h5 style="color: pink;">KNN</h5>

In [254]:
# KNN

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

80.72

<h5 style="color: pink;">Gaussian Naive Bayes</h5>

In [255]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

40.0

<h5 style="color: pink;">Perceptron</h5>

In [256]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

36.09

<h5 style="color: pink;">Linear SVC</h5>

In [257]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

C:\Users\marko\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


39.13

<h5 style="color: pink;">Stochastic Gradient Descent</h5>

In [258]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

35.65

<h5 style="color: pink;">Decision Tree</h5>

In [259]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

<h5 style="color: pink;">Random Forest</h5>

In [260]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest


100.0

La lunghezza dei df: il primo è il numero degli elementi all'interno del dataframe di test, mentre il secondo è il numero di elementi predetti che sono uguali al risultato effettivo di test, dunque sono predict corrette.

In [261]:
X_test['Y_predict'] = Y_pred
X_test['Y'] = Y_test
print(len(X_test.index))
print(len(X_test[X_test['Y_predict'] == X_test['Y']].index))

1608
1203


<h3 style="color: gold;">Risultati:</h3>

<p style="color: gray;">Ora stampo le varie percentuali di accuratezza per vedere quale modello si adatta meglio allo studio che voglio fare, ordinandoli in ordine decresente per percentuale</p>

In [262]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,100.00
8,Decision Tree,100.00
1,KNN,80.72
2,Logistic Regression,45.09
4,Naive Bayes,40.00
0,Support Vector Machines,39.28
7,Linear SVC,39.13
5,Perceptron,36.09
6,Stochastic Gradient Decent,35.65


<p style ="color: gray;">Dalla tabella che ne risulta abbiamo che Random Forest e Decion Tree sono i modelli più accurati nel predirre la gravità delle condizioni del paziente.</p>